In [121]:
import pandas as pd
from category_encoders.target_encoder import TargetEncoder
from catboost import CatBoostClassifier


In [78]:
def normalize(df_train, df_test, mode: str = 'mean'):
    if mode == 'mean':
        func = lambda df: (df - df.mean())/df.std()
    elif mode == 'mm':
        func = lambda df: (df-df.min())/(df.max()-df.min())
    return (func(df_train), func(df_test))

In [122]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [123]:
df_train

,id,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,rating,city_code,region_code,center_type,op_area,category,cuisine,many_orders
0,1379560,55,1885,136.83,152.29,0,0,340,647,56,TYPE_C,2.0,Beverages,Thai,1
1,1466964,55,1993,136.83,135.83,0,0,471,647,56,TYPE_C,2.0,Beverages,Thai,1
2,1346989,55,2539,134.86,135.86,0,0,549,647,56,TYPE_C,2.0,Beverages,Thai,1
3,1338232,55,2139,339.50,437.53,0,0,298,647,56,TYPE_C,2.0,Beverages,Indian,0
4,1448490,55,2631,243.50,242.50,0,0,219,647,56,TYPE_C,2.0,Beverages,Indian,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305983,1475937,61,1543,472.39,473.39,0,0,169,473,77,TYPE_A,4.5,Desert,Indian,0
305984,1143788,61,2304,472.39,473.39,0,0,228,473,77,TYPE_A,4.5,Desert,Indian,0
305985,1096621,61,2664,311.43,310.43,0,0,556,473,77,TYPE_A,4.5,Salad,Italian,1
305986,1456106,61,2569,310.43,309.43,0,0,773,473,77,TYPE_A,4.5,Salad,Italian,1


In [124]:
df_test

,id,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,rating,city_code,region_code,center_type,op_area,category,cuisine
0,1256496,55,1885,144.53,146.53,0,0,3240,647,56,TYPE_C,2.0,Beverages,Thai
1,1298009,55,1993,149.44,149.44,0,0,914,647,56,TYPE_C,2.0,Beverages,Thai
2,1348875,55,2539,150.35,152.35,0,0,4104,647,56,TYPE_C,2.0,Beverages,Thai
3,1272771,55,2139,307.49,307.49,0,0,1970,647,56,TYPE_C,2.0,Beverages,Indian
4,1313265,55,2631,97.06,154.26,0,0,2900,647,56,TYPE_C,2.0,Beverages,Indian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150555,1271326,61,1543,484.09,484.09,0,0,1674,473,77,TYPE_A,4.5,Desert,Indian
150556,1062036,61,2304,482.09,482.09,0,0,1899,473,77,TYPE_A,4.5,Desert,Indian
150557,1110849,61,2664,237.68,321.07,0,0,1445,473,77,TYPE_A,4.5,Salad,Italian
150558,1147725,61,2569,243.50,313.34,0,0,3209,473,77,TYPE_A,4.5,Salad,Italian


In [125]:
df_train.drop(['rating'], axis = 1, inplace=True)
df_test.drop(['rating'], axis = 1, inplace = True)

In [110]:
enc = TargetEncoder()
enc.fit(df_train[['center_type', 'category', 'cuisine']], df_train['many_orders'])
df_train[['center_type', 'category', 'cuisine']] = enc.transform(df_train[['center_type', 'category', 'cuisine']])

In [111]:
df_test[['center_type', 'category', 'cuisine']] = enc.transform(df_test[['center_type', 'category', 'cuisine']])

In [112]:
#Нормализация
df_train.loc[:, 'center_id':'cuisine'], df_test.loc[:, 'center_id':'cuisine'] = normalize(df_train.loc[:, 'center_id':'cuisine'], df_test.loc[:, 'center_id':'cuisine'], mode = 'mean')

In [126]:
model = CatBoostClassifier(cat_features = ['center_type', 'category', 'cuisine'])

In [127]:
X = df_train.loc[:, 'center_id':'cuisine']
y = df_train['many_orders']

model_fitted = model.fit(X, y)

Learning rate set to 0.118667
0:	learn: 0.6183748	total: 130ms	remaining: 2m 9s
1:	learn: 0.5617700	total: 241ms	remaining: 2m
2:	learn: 0.5203531	total: 360ms	remaining: 1m 59s
3:	learn: 0.4859710	total: 449ms	remaining: 1m 51s
4:	learn: 0.4627325	total: 580ms	remaining: 1m 55s
5:	learn: 0.4398538	total: 672ms	remaining: 1m 51s
6:	learn: 0.4258707	total: 794ms	remaining: 1m 52s
7:	learn: 0.4135791	total: 888ms	remaining: 1m 50s
8:	learn: 0.4027025	total: 981ms	remaining: 1m 48s
9:	learn: 0.3937126	total: 1.09s	remaining: 1m 48s
10:	learn: 0.3883154	total: 1.29s	remaining: 1m 56s
11:	learn: 0.3829292	total: 1.41s	remaining: 1m 56s
12:	learn: 0.3777787	total: 1.54s	remaining: 1m 57s
13:	learn: 0.3733077	total: 1.65s	remaining: 1m 56s
14:	learn: 0.3685469	total: 1.83s	remaining: 2m
15:	learn: 0.3648483	total: 1.93s	remaining: 1m 58s
16:	learn: 0.3620046	total: 2.08s	remaining: 2m
17:	learn: 0.3596053	total: 2.23s	remaining: 2m 1s
18:	learn: 0.3568456	total: 2.38s	remaining: 2m 3s
19:	lea

In [139]:
proba_predicts = model_fitted.predict_proba(df_test.loc[:, 'center_id':])

In [140]:
df_res = pd.DataFrame()
df_res['id'] = df_test['id']
df_res['many_orders'] = list(proba_predicts[:, 1])

In [141]:
df_res

,id,many_orders
0,1256496,0.879733
1,1298009,0.897472
2,1348875,0.755116
3,1272771,0.001055
4,1313265,0.077478
...,...,...
150555,1271326,0.004368
150556,1062036,0.002869
150557,1110849,0.995685
150558,1147725,0.975174


In [118]:
df_res.to_csv('submission.csv', index = False)